In [1]:
import pandas as pd
import sys
import glob
import os

from tqdm import tqdm

sys.path.append("../../../../utils")
from absolute_path_builder import AbsolutePathBuilder

In [2]:
SCORE_MODEL = "flair"

youtube_path = AbsolutePathBuilder.get_path(
    f"06_youtube_regression",
    filepaths="../../../../config/filepaths.yaml"
)

coraal_vs_buckeye_path = AbsolutePathBuilder.get_path(
    f"06_coraal_vs_buckeye_regression",
    filepaths="../../../../config/filepaths.yaml"
)

twitter_path = AbsolutePathBuilder.get_path(
    f"06_twitter_regression",
    filepaths="../../../../config/filepaths.yaml"
)

In [10]:
dfs = []

i = 0
for input_path in [
    youtube_path,
    coraal_vs_buckeye_path,
    twitter_path
]:
    df = pd.read_csv(os.path.join(input_path, f"{SCORE_MODEL}_score_coefs.csv"))
    df["base"] = i
    dfs.append(df)
    
    i += 1
    
df = pd.concat(dfs)

In [11]:
pd.set_option('display.max_rows', None)

In [13]:
cols = df[df.pvalue < 0.01].column.unique()

df[df.column.isin(cols)].sort_values("column").shape

(195, 4)

In [7]:
df = df.groupby("column", as_index=False).mean()

In [8]:
df[df.pvalue < 0.05]

,column,pvalue,coef,base
11,LIWC_BIO,0.000258,-10.953950,1.0
12,LIWC_BODY,0.000354,6.866180,1.0
14,LIWC_CERTAIN,0.006554,-2.300069,1.0
28,LIWC_FOCUSPRESENT,0.023617,-3.220721,1.0
31,LIWC_HEALTH,0.000516,6.909265,1.0
36,LIWC_INGEST,0.004162,8.859609,1.0
41,LIWC_MALE,0.026650,0.212332,1.0
44,LIWC_NEGATE,0.005302,7.181092,1.0
62,LIWC_SEXUAL,0.000190,6.201504,1.0
70,LIWC_VERB,0.015549,3.851578,1.0
